## PLS example

The FHI project uses Partial least squares regression (PLS regression) to try and predict drug concentrations.
It uses image segmentation for the data and Works with this R project to generate PLSR coefficients https://github.com/sortijas/PAD . 

In this example we will download images from the ```FHI2022``` project and calculate their concentrations.

The workflow is
1. Grab some PLS coefficients. These can be generated by the R program or downloaded from the PADs website.
1. Download and loop through the images and run the image segmentation/PLS prediction code.

## PLS Prediction

### Load data to be predicted

In [1]:
import sys, os
import pandas as pd
sys.path.append(os.path.abspath('../src'))
import pad_helper 

def standardize_names(name):
    return name.lower().replace(' ', '-')

# set url for pad server
pad_url = 'https://pad.crc.nd.edu/'

# you need to ask Chris nicely for an API key
API_KEY = '5NWT4K7IS60WMLR3J2LV'

# lets grab some data, in the example we download all records for project "FHI2022"
all_data = pad_helper.query_pad_database("FHI2020", API_KEY)

# check if we were succesful
if all_data and 'status' in all_data:
    if all_data['status'] == 'ko':
        print("Error:", all_data['error_description'])
    else:
        # if succesful we can create a pandas table
        if 'data' in all_data:
            df = pd.DataFrame(all_data['data'])
            if 'headers' in all_data:
                df.columns = all_data['headers']
            print("Data loaded!")
        else:
            print("Empty query!")


Data loaded!


### Standardize API names

In [3]:
df['sample_name'] = df['sample_name'].apply(standardize_names)   

###  Pre-Process Test Data 

> Run the data segmentation for getting the rgb values per line or load it from a file.


#### Load a file with the  rgb values per line 

In [4]:
# Load the test data
rgb_fpath = '../data/10_region_rgb__test.csv'

# load the test data
rgb_values_df = pd.read_csv(rgb_fpath)

# Mapping the test data accordingly with the data present in the file with rgb values
test_df = df[df['id'].isin(rgb_values_df['Image'])]


In [72]:
# list blank and swiped-but-not-run (not originally in the data from FHI2022)
# rgb_values_df['Contains'][~rgb_values_df['Image'].isin(df['id'])].value_counts()

#### Create dat structure for saving results

In [5]:
# Creating the 'results_pls' dataframe by selecting relevant columns from 'test_df' and adding 'pred_quantity'
results_pls = pd.DataFrame({
    'id': test_df['id'],
    'sample_id': test_df['sample_id'],
    'actual_class': test_df['sample_name'],  # Renaming 'sample_name' to 'actual_class'
    'actual_quantity': test_df['quantity'],  # Renaming 'quantity' to 'actual_quantity'
    'pred_quantity': [None] * len(test_df)   # Initializing 'pred_quantity' with None values
})

results_pls

,id,sample_id,actual_class,actual_quantity,pred_quantity
18,15229,53848,amoxicillin,100,None
22,15233,53697,amoxicillin,100,None
25,15236,53703,amoxicillin,100,None
27,15238,53703,amoxicillin,100,None
37,15253,53712,amoxicillin,80,None
...,...,...,...,...,...
9669,25637,55075,ripe,50,None
9670,25638,55492,ripe,20,None
9674,25642,55075,ripe,50,None
9678,25646,55432,ripe,80,None


### Load the PLS coefficients

In [6]:
# Download the PLS coefficients
pls_url = 'https://pad.crc.nd.edu/neuralnetworks/pls/24fhiPLS1quantity/1.0/24fhiPLS1quantity.csv'

# call helper function
if pad_helper.pad_download(pls_url):
    print(pls_url, "downloaded.")

https://pad.crc.nd.edu/neuralnetworks/pls/24fhiPLS1quantity/1.0/24fhiPLS1quantity.csv downloaded.


#### PLS class instance

In [7]:
import csv
import cv2 as cv
import numpy as np
import urllib3
from PIL import Image, ImageFile
import regionRoutine

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
ImageFile.LOAD_TRUNCATED_IMAGES = True

def download_file(url, filename, images_path):
    """Download a file from a URL and save it to a local file."""
    try:
        response = requests.get(url, stream=True, verify=False)
        if response.status_code == 200:
            path = os.path.join(images_path, filename)
            with open(path, 'wb') as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
            # print(f"File '{filename}' successfully downloaded to '{images_path}'")
        else:
            # Log error if the response status code is not 200
            print(f"Failed to download the file. URL: {url} returned status code: {response.status_code}")
            raise Exception(f"Failed to download the file. URL: {url} returned status code: {response.status_code}")
    except Exception as e:
        # Log any other exceptions during the download process
        print(f"An error occurred while downloading the file: {e}")
        # Optionally, you can re-raise the exception if you want it to be noticed by the calling function
        raise

def convert_from_image_to_cv2(img: Image) -> np.ndarray:
    # return np.asarray(img)
    return cv.cvtColor(np.array(img), cv.COLOR_RGB2BGR)

class pls:
    def __init__(self, coefficients_file):
        try:
            # load coeffs
            self.coeff = {}
            with open(coefficients_file) as csvcoeffs:
                csvcoeffreader = csv.reader(csvcoeffs)
                #i=0
                for row in csvcoeffreader:
                    elmts = []
                    for j in range(1,len(row)):
                        elmts.append(float(row[j]))
                    self.coeff[row[0]] = elmts
        except Exception as e:
            print("Error",e, "loading pls coefficients", coefficients_file)

    def quantity(self, in_file, drug):
        try:
            # grab image
            img = cv.imread(in_file)
            
            if img is None:
                print("Converting img.. ", in_file)
                # read image using Pillow and covert to cv2
                img_pil = Image.open(in_file)
                img = convert_from_image_to_cv2(img_pil)
            
            if img is None:
                raise Exception(f"Failed to load the file. URL: {in_file}.") 

            # pls dictionary
            f = {}
            f = regionRoutine.fullRoutine(img, regionRoutine.intFind.findMaxIntensitiesFiltered, f, True, 10)

            # drug?
            # continue if no coefficients
            
            if drug.lower() not in self.coeff:
                print(drug.lower(), "--- NOT IN COEFFICIENTS FILE ---")
                return -1
            
            

            drug_coeff = self.coeff[drug.lower()] #coeff['amoxicillin'] #

            # start with offst
            pls_concentration = drug_coeff[0]

            coeff_index = 1

            for letter in ['A','B','C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L']:
                for region in range(10):
                    for color_letter in ['R', 'G', 'B']:
                        pixval = f[letter + str(region + 1) + '-' + color_letter]
                        pls_concentration += float(pixval) * drug_coeff[coeff_index]
                        coeff_index += 1
                        
            # print(drug.lower(), "--- OK ---")
            return pls_concentration
        
        except Exception as e:
            print("Error",e, "pls analyzing image", in_file, "with", drug)
            return -1.
        
        
# for api in sample_names:
#     if api.lower() in pls_conc.coeff:
#         print(api, pls_conc.coeff[api.lower()])
#     else:
#         print(api.lower(), 0.0)


# def fix_img(name, img_dir):
#     img_path = os.path.join(img_dir, name)
#     img_pil = Image.open(img_path)
#     img_pil.save(img_path)     

In [8]:
# STAGE 2
import os, requests

# Create a PLS class instance
import pad_analysis

# name 
pls_file = os.path.basename(pls_url)
print(pls_file)

# creat pls instance
pls_conc = pls(pls_file)

2024-11-15 10:25:04.214594: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-15 10:25:04.265332: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-15 10:25:04.265371: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-15 10:25:04.265401: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 10:25:04.275038: I tensorflow/core/platform/cpu_feature_g

24fhiPLS1quantity.csv


#### Get nn_pred_api from NN results

In [10]:
test_df_nn_pred = pd.read_csv('../data/final_results__nn-all.csv')
test_df_nn_pred['nn_pred_api'] = test_df_nn_pred['nn_pred_api'].apply(standardize_names)

In [11]:
import pandas as pd
from tqdm import tqdm

# Assume test_df is already defined as a pandas dataframe
# Assume download_file and pls_conc.quantity are predefined functions

# Define temporary file and output file for intermediate values
temp_file = './temp.png'
intermediate_file = '../data/apagar_pls_2_intermediate_results.csv'


# Open the intermediate file to write intermediate results
with open(intermediate_file, 'w') as f:
    # Write header row for the intermediate CSV
    f.write('count,index,id,sample_id,sample_name, actual_sample_name,pred_quantity,actual_quantity\n')

    # Count the rows to limit the loop
    row_count = 0

    # Loop over rows with progress bar
    for index, row_data in tqdm(test_df.iterrows(), total=len(test_df)):
        # Update counter
        row_count += 1

        # Get image location
        image_url = pad_url + str(row_data['processed_file_location'])

        # Download it to a temporary file
        download_file(image_url, temp_file, './')

        # Analyze it to get predicted concentration
        sample_name = test_df_nn_pred.loc[test_df_nn_pred.id == row_data['id'], 'nn_pred_api'].values[0]
        pls_concentration = pls_conc.quantity(temp_file, sample_name)

        # Print results (optional)
        # print("PAD", row_data['id'], row_data['sample_id'], row_data['nn_pred_api'], 
        #       "PLS concentration", "{:.1f}".format(pls_concentration), "%, actual", row_data['quantity'])

        # Save intermediate values to the file
        f.write(f"{row_count-1},{index},{row_data['id']},{row_data['sample_id']},{sample_name}, {row_data['sample_name']},{pls_concentration},{row_data['quantity']}\n")

        # Flush to ensure the data is saved to the file
        f.flush()
        
        # # Break once we have our 10 data points (if required)
        # if row_count >= 3:
        #     break



 22%|██▏       | 441/2000 [04:27<15:13,  1.71it/s]

lactose --- NOT IN COEFFICIENTS FILE ---


 22%|██▏       | 442/2000 [04:27<15:09,  1.71it/s]

lactose --- NOT IN COEFFICIENTS FILE ---


 22%|██▏       | 443/2000 [04:28<14:55,  1.74it/s]

...

 99%|█████████▊| 1972/2000 [19:36<00:16,  1.72it/s]libpng error: Read Error


Converting img..  ./temp.png


100%|█████████▉| 1992/2000 [19:47<00:04,  1.73it/s]libpng error: Read Error


Converting img..  ./temp.png


100%|██████████| 2000/2000 [19:52<00:00,  1.68it/s]


In [12]:

# Read the intermediate file into a dataframe
intermediate_df = pd.read_csv(intermediate_file)

# Use 'id' as the key to update the results_pls dataframe with predicted quantities
# Assuming that results_pls already contains the ids, sample_id, and sample_name
results_pls.set_index('id', inplace=True)
intermediate_df.set_index('id', inplace=True)

# Update only the 'pred_quantity' column in results_pls where 'id' matches
results_pls.update(intermediate_df[['pred_quantity']])

# Reset index after update (if needed)
results_pls.reset_index(inplace=True)

# Save the final results to a CSV file
results_pls.to_csv('../data/apagar_final-results__nn-api__pls-quantity.csv', index=False)

In [13]:
results_pls

,id,sample_id,actual_class,actual_quantity,pred_quantity
0,15229,53848,amoxicillin,100,62.669838
1,15233,53697,amoxicillin,100,107.374761
2,15236,53703,amoxicillin,100,58.5943
3,15238,53703,amoxicillin,100,60.836433
4,15253,53712,amoxicillin,80,77.704915
...,...,...,...,...,...
1995,25637,55075,ripe,50,39.240428
1996,25638,55492,ripe,20,45.20178
1997,25642,55075,ripe,50,44.858634
1998,25646,55432,ripe,80,93.184574


In [14]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Calculate the RMSE between actual_quantity and pred_quantity
rmse = np.sqrt(mean_squared_error(results_pls['actual_quantity'].astype(int), results_pls['pred_quantity'].astype(int)))

# Print the RMSE
print(f"RMSE: {rmse}")

RMSE: 20.560617695001284


In [49]:
# Exclude 'lactose' from the RMSE calculation
filtered_results_pls = results_pls[results_pls['actual_class'] != 'lactose']

# Calculate the RMSE between actual_quantity and pred_quantity for the filtered data
rmse_filtered = np.sqrt(mean_squared_error(filtered_results_pls['actual_quantity'].astype(int), filtered_results_pls['pred_quantity'].astype(int)))

# Print the RMSE for the filtered data
print(f"RMSE (excluding 'lactose'): {rmse_filtered}")

RMSE (excluding 'lactose'): 18.273899933208572


In [45]:
# Function to calculate RMSE
def calculate_rmse(group, pred_col='pred_quantity', actual_col='actual_quantity'):
    actual = group[actual_col].astype(int)
    predicted = group[pred_col].astype(int)
    return np.sqrt(np.mean((actual - predicted) ** 2))


In [46]:
# Grouping by 'actual_class' and applying the RMSE calculation
rmse_by_class = results_pls.groupby('actual_class').apply(calculate_rmse)

# Display the RMSE for each actual_class
display(rmse_by_class)

/tmp/ipykernel_1207684/3144179928.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  rmse_by_class = results_pls.groupby('actual_class').apply(calculate_rmse)


actual_class
albendazole                    26.696903
amoxicillin                    21.116057
ampicillin                     21.515612
azithromycin                   16.158185
benzyl-penicillin              25.205311
ceftriaxone                    15.350738
chloroquine                    16.784456
ciprofloxacin                  23.281069
doxycycline                    18.537720
epinephrine                    17.309887
ethambutol                     17.032209
ferrous-sulfate                17.331646
hydroxychloroquine             14.861458
isoniazid                      15.019094
lactose                       101.000000
promethazine-hydrochloride     22.108398
pyrazinamide                   17.776891
rifampicin                     19.451706
ripe                           15.096387
sulfamethoxazole               14.472270
tetracycline                   16.621427
dtype: float64

In [47]:
rmse_by_class.to_csv('../data/rmse_by_class__nn-api__pls-quantity.csv')